In [1]:
import spacy
import pandas as pd

In [2]:
df = pd.read_excel('../../data/raw/OMIEC_NEUROMORPHIC_07_08_24.xls')

In [3]:
texts = df['Abstract']

# Carregar o modelo de linguagem
nlp = spacy.load("en_core_web_sm")

# Processar os textos com apenas o tokenizer ativo
for i, doc in enumerate(nlp.pipe(texts, n_process=2,disable=["tok2vec", "tagger", "parser", "attribute_ruler", "ner"])):
    # Conversão para minúsculas, remoção de stop words e pontuação
    tokens = [token.text.lower() for token in doc if not token.is_stop and not token.is_punct]
    print(f"Index {i} - {tokens}")


Index 0 - ['field', 'organic', 'mixed', 'ionic', 'electronic', 'conductors', 'omiecs', 'gained', 'significant', 'attention', 'ability', 'transport', 'electrons', 'ions', 'making', 'promising', 'candidates', 'applications', 'initially', 'focused', 'inorganic', 'materials', 'exploration', 'mixed', 'conduction', 'expanded', 'organic', 'materials', 'especially', 'polymers', 'owing', 'advantages', 'solution', 'processability', 'flexibility', 'property', 'tunability', 'omiecs', 'particularly', 'form', 'polymers', 'possess', 'electronic', 'ionic', 'transport', 'functionalities', 'review', 'provides', 'overview', 'omiecs', 'aspects', 'covering', 'mechanisms', 'charge', 'transport', 'including', 'electronic', 'transport', 'ionic', 'transport', 'ionic', 'electronic', 'coupling', 'conducting', 'semiconducting', 'conjugated', 'polymers', 'applications', 'organic', 'bioelectronics', 'including', 'multi)sensors', 'neuromorphic', 'devices', 'electrochromic', 'devices', 'omiecs', 'promise', 'organic',

#### Playground

In [4]:
import spacy

texts = [
    "Net income was $9.4 million compared to the prior year of $2.7 million.",
    "Revenue exceeded twelve billion dollars, with a loss of $1b.",
]

# Carregar o modelo de linguagem
nlp = spacy.load("en_core_web_sm")

# Processar os textos com apenas o tokenizer ativo
for i, doc in enumerate(nlp.pipe(texts, n_process=2,disable=["tok2vec", "tagger", "parser", "attribute_ruler", "ner"])):
    # Conversão para minúsculas, remoção de stop words e pontuação
    tokens = [token.text.lower() for token in doc if not token.is_stop and not token.is_punct]
    print(f"Index {i} - {tokens}")


Index 0 - ['net', 'income', '$', '9.4', 'million', 'compared', 'prior', 'year', '$', '2.7', 'million']
Index 1 - ['revenue', 'exceeded', 'billion', 'dollars', 'loss', '$', '1b']
